In [1]:
import openai
import langchain
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI

d:\LangchainPractice\PDFReaderPineconeVectorDB\pineconeEnv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os

In [4]:
# Reading the document...
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    document_file = file_loader.load()
    return document_file

In [5]:
doc = read_doc("Document/")
len(doc) # No. of pages in the PDF.

58

In [6]:
## Divide the docs into chunks
def chunk_data(docs, chunk_size=800, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    doc = text_splitter.split_documents(docs)
    return doc

In [7]:
documents=chunk_data(docs=doc)
len(documents)

141

In [8]:
# Embedding techniques of OpenAI
embeddings = OpenAIEmbeddings(api_key=os.environ["OPENAI_API_KEY"])
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000002377A9B7D30>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000002377A988730>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [9]:
vectors=embeddings.embed_query("How are you? I am fine thankyou")
len(vectors)

1536

In [11]:
import os 
os.environ['PINECONE_API_KEY'] = ""

In [13]:
index_name = "langchainvector"

In [14]:
index=Pinecone.from_documents(documents=doc,embedding=embeddings,index_name=index_name)

In [15]:
## Cosine Similarity Retreive Results from VectorDB
def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [16]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [17]:
llm=OpenAI(temperature=0.5)
chain=load_qa_chain(llm,chain_type="stuff")

d:\LangchainPractice\PDFReaderPineconeVectorDB\pineconeEnv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [18]:
## Search answers from VectorDB
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [19]:
our_query = "How much the agriculture target will be increased by how many crore?"
answer = retrieve_answers(our_query)
print(answer)

[Document(page_content="7 \n \n \n farmers in contributing to the health of fellow citizens by growing these \n‘Shree Anna’.   \n22. Now to make India a global hub for ' Shree Anna' , the Indian Institute \nof Millet Research, Hyderabad  will be supported as the Centre of Excellence \nfor sharing best practices, research and technologies at the international \nlevel.    \nAgriculture Credit  \n23. The agriculture credit target will be increased  \nto ` 20 lakh crore with focus on animal husbandry, dairy and fisheries.  \nFisheries \n24. We will launch a new sub-scheme of PM Matsya Sampada Yojana \nwith targeted investment of ` 6,000 crore to further enable activities of \nfishermen, fish vendors, and micro & small enterprises, improve value chain \nefficiencies, and expand the market. \nCooperation \n25. For farmers, especially small and marginal farmers, and other \nmarginalised sections, the government is promoting cooperative-based \neconomic development model. A new Ministry of Coo

d:\LangchainPractice\PDFReaderPineconeVectorDB\pineconeEnv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 The agriculture credit target will be increased to ` 20 lakh crore.
